In [1]:
import os
import sys
import torch
from datasets import load_from_disk
from datasets import DatasetDict
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import AutoModelForSeq2SeqLM
from peft import LoraConfig, get_peft_model, prepare_model_for_int8_training, TaskType
from transformers import DataCollatorForSeq2Seq
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments


os.environ["CUDA_VISIBLE_DEVICES"] = "2,3"

In [2]:
print(sys.version)

3.9.18 (main, Sep 11 2023, 13:41:44) 
[GCC 11.2.0]


In [3]:

# Check if CUDA (GPU support) is available
if torch.cuda.is_available():
    num_gpus = torch.cuda.device_count()
    for i in range(num_gpus):
        print(f"GPU {i}: {torch.cuda.get_device_name(i)}")
else:
    print("No GPU available, using CPU instead.")


GPU 0: NVIDIA GeForce RTX 3090
GPU 1: NVIDIA GeForce RTX 3090


In [ ]:
dataset = load_from_disk('./dataset/hf-dataset')

In [6]:
training = load_from_disk('./data/train')

In [7]:
validation = load_from_disk('./data/validation')

In [8]:
testing = load_from_disk('./data/eval')

In [9]:
tokenized_dataset = DatasetDict({
    "train": training,
    "validation": validation,
    "test": testing,
})

In [31]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label', '__index_level_0__'],
        num_rows: 20000
    })
    validation: Dataset({
        features: ['text', 'label', '__index_level_0__'],
        num_rows: 4000
    })
    test: Dataset({
        features: ['text', 'label', '__index_level_0__'],
        num_rows: 2000
    })
})

In [11]:
model_id="google/mt5-xl"

In [12]:
tokenizer = AutoTokenizer.from_pretrained(model_id)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/home/pricie/cclstudent6/tanmay-thesis/thesis/lib/python3.9/site-packages/transformers/convert_slow_tokenizer.py:550: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [25]:
def count_tokens_over_512(split):
    count = 0
    i = 0
    for example in dataset[split]:
        tokenized_text = tokenizer(example["text"], return_tensors="pt")['input_ids']
        if tokenized_text.size(1) > 512:
            count += 1
            
    return count
train_count = count_tokens_over_512("train")
validation_count = count_tokens_over_512("validation")
test_count = count_tokens_over_512("test")
print("Train:", train_count)
print("Validation:", validation_count)
print("Test:", test_count)

Train: 0
Validation: 0
Test: 0


In [39]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label', '__index_level_0__'],
        num_rows: 20000
    })
    validation: Dataset({
        features: ['text', 'label', '__index_level_0__'],
        num_rows: 4000
    })
    test: Dataset({
        features: ['text', 'label', '__index_level_0__'],
        num_rows: 2000
    })
})

In [41]:
def preprocess_function(sample, padding="max_length"):
    inputs = sample["text"]
    model_inputs = tokenizer(inputs, max_length=512, padding=padding, truncation=True)
    labels = tokenizer(text_target=sample["label"], max_length=200, padding=padding, truncation=True)
    if padding == "max_length":
        labels["input_ids"] = [
            [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
        ]
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs
tokenized_dataset = dataset.map(
    lambda x: preprocess_function(x),
    batched=True,
    remove_columns=["text", "label"]
)
print(f"Keys of tokenized dataset: {list(tokenized_dataset['train'].features)}")
# Save datasets to disk for later easy loading
tokenized_dataset["train"].save_to_disk("data/train")
tokenized_dataset["validation"].save_to_disk("data/validation")
tokenized_dataset["test"].save_to_disk("data/eval")


Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Keys of tokenized dataset: ['__index_level_0__', 'input_ids', 'attention_mask', 'labels']


Saving the dataset (0/1 shards):   0%|          | 0/20000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/4000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/2000 [00:00<?, ? examples/s]

In [14]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_id, load_in_8bit=True, device_map="auto")

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


In [16]:
model

MT5ForConditionalGeneration(
  (shared): Embedding(250112, 2048)
  (encoder): MT5Stack(
    (embed_tokens): Embedding(250112, 2048)
    (block): ModuleList(
      (0): MT5Block(
        (layer): ModuleList(
          (0): MT5LayerSelfAttention(
            (SelfAttention): MT5Attention(
              (q): Linear8bitLt(in_features=2048, out_features=2048, bias=False)
              (k): Linear8bitLt(in_features=2048, out_features=2048, bias=False)
              (v): Linear8bitLt(in_features=2048, out_features=2048, bias=False)
              (o): Linear8bitLt(in_features=2048, out_features=2048, bias=False)
              (relative_attention_bias): Embedding(32, 32)
            )
            (layer_norm): MT5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): MT5LayerFF(
            (DenseReluDense): MT5DenseGatedActDense(
              (wi_0): Linear8bitLt(in_features=2048, out_features=5120, bias=False)
              (wi_1): Linear8bitLt(in_feature

In [17]:
lora_config = LoraConfig(
 r=8,
 lora_alpha=32,
 target_modules="all-linear",
 lora_dropout=0.05,
 bias="none",
 task_type=TaskType.SEQ_2_SEQ_LM
)

In [18]:
model = prepare_model_for_int8_training(model)

/home/pricie/cclstudent6/tanmay-thesis/thesis/lib/python3.9/site-packages/peft/utils/other.py:145: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


In [19]:
model = get_peft_model(model, lora_config)

In [20]:
model.print_trainable_parameters()

trainable params: 17,694,720 || all params: 3,760,314,368 || trainable%: 0.47056491208769063


In [22]:
label_pad_token_id = -100

In [23]:
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=label_pad_token_id,
    pad_to_multiple_of=8
)

In [25]:
output_dir="lora-mt5-xl"

In [26]:
import wandb
wandb.login()

wandb: Currently logged in as: tanmay-k (team-tk). Use `wandb login --relogin` to force relogin


True

In [27]:
os.environ["WANDB_PROJECT"]="thesis-lora-mt5-xl"

In [28]:
os.environ["WANDB_LOG_MODEL"] = "checkpoint"

In [29]:
os.environ["WANDB_PROJECT"]

'thesis-lora-mt5-xl'

In [30]:
last_run_id = "u4l9ezqa"

In [31]:
api = wandb.Api()
runs = api.runs('thesis-lora-mt5-xl')
for run in runs:
  print(run.id)

u4l9ezqa


In [32]:
for run in runs:
  print(f"Run ID: {run.id}")
  if run.config and 'model' in run.config and 'name' in run.config.model:
    model_name = run.config.model.name
    print(f"  Model Name: {model_name}")
  else:
    print(f"  Model Name: Not found in run config")

Run ID: u4l9ezqa
  Model Name: Not found in run config


In [33]:
with wandb.init(
    project=os.environ["WANDB_PROJECT"],
    id=last_run_id,
    resume="must",
) as run:
    my_checkpoint_name = f"checkpoint-{last_run_id}:latest"
    my_checkpoint_artifact = run.use_artifact('team-tk/thesis-lora-mt5-xl/checkpoint-u4l9ezqa:v0', type='model')
    checkpoint_dir = my_checkpoint_artifact.download()
    training_args = Seq2SeqTrainingArguments(
    output_dir=output_dir,
	auto_find_batch_size=True,
    learning_rate=1e-3,
    num_train_epochs=5,
    logging_dir=f"{output_dir}/logs",
    logging_strategy="steps",
    logging_steps=500,
    save_strategy="epoch",
    report_to="wandb",
    )
    trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"]
    )
    trainer.train(resume_from_checkpoint=checkpoint_dir)

wandb: Downloading large artifact checkpoint-u4l9ezqa:v0, 203.34MB. 8 files... 
wandb:   8 of 8 files downloaded.  
Done. 0:0:4.1
/home/pricie/cclstudent6/tanmay-thesis/thesis/lib/python3.9/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
/home/pricie/cclstudent6/tanmay-thesis/thesis/lib/python3.9/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentr

Step,Training Loss
3000,2.539000
3500,2.560200
4000,2.558900
4500,3.148100
5000,3.143100
5500,2.973700
6000,2.916000
6500,2.825600
7000,2.872500
7500,2.869600


wandb: Adding directory to artifact (./lora-mt5-xl/checkpoint-5000)... Done. 2.1s
/home/pricie/cclstudent6/tanmay-thesis/thesis/lib/python3.9/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/home/pricie/cclstudent6/tanmay-thesis/thesis/lib/python3.9/site-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
wandb: Adding directory to artifact (./lora-mt5-xl/checkpoint-7500)... Done. 2.1s
/home/pricie/cclstudent6/tanmay-the

wandb: WARNING Ensure read and write access to run files dir: /home/pricie/cclstudent6/tanmay-thesis/wandb/run-20240315_000413-u4l9ezqa/files, control this via the WANDB_DIR env var. See https://docs.wandb.ai/guides/track/environment-variables


train/epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇███
train/global_step,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇███
train/grad_norm,▄▃▂▁▁▁▁▄▁▁▁▁▁▂▆█▃▇▁▄
train/learning_rate,██▇▇▇▆▆▅▅▅▄▄▄▃▃▂▂▂▁▁
train/loss,▃▃▃██▇▆▅▆▆▄▄▄▃▃▂▁▁▁▁
train/total_flos,▁
train/train_loss,▁
train/train_runtime,▁
train/train_samples_per_second,▁
train/train_steps_per_second,▁
train/epoch,5.0


In [ ]:
wandb.sync()

In [53]:
wandb.finish()

In [60]:
training_args = Seq2SeqTrainingArguments(
    output_dir=output_dir,
	auto_find_batch_size=True,
    learning_rate=1e-3,
    num_train_epochs=5,
    logging_dir=f"{output_dir}/logs",
    logging_strategy="steps",
    logging_steps=500,
    save_strategy="epoch",
    report_to="wandb",
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"]
)

/home/pricie/cclstudent6/tanmay-thesis/thesis/lib/python3.9/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [ ]:
trainer.train()
wandb.finish()

wandb: Currently logged in as: tanmay-k (team-tk). Use `wandb login --relogin` to force relogin
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


/home/pricie/cclstudent6/tanmay-thesis/thesis/lib/python3.9/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/home/pricie/cclstudent6/tanmay-thesis/thesis/lib/python3.9/site-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


Step,Training Loss
500,2.896800
1000,2.718000
1500,2.731800
2000,2.700500
2500,2.682600
3000,2.570900
3500,2.633200
4000,2.616700
4500,2.611500


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
wandb: Adding directory to artifact (./lora-mt5-xl/checkpoint-2500)... Done. 2.1s
/home/pricie/cclstudent6/tanmay-thesis/thesis/lib/python3.9/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/home/pricie/cclstudent6/tanmay-thesis/thesis/lib/python3.9/site-packages/bitsandbytes/autograd/_function

In [34]:
model.save_pretrained("./model_backup/lora_mt5_xl")
tokenizer.save_pretrained("./model_backup/lora_mt5_xl")

('./model_backup/lora_mt5_xl/tokenizer_config.json',
 './model_backup/lora_mt5_xl/special_tokens_map.json',
 './model_backup/lora_mt5_xl/spiece.model',
 './model_backup/lora_mt5_xl/added_tokens.json',
 './model_backup/lora_mt5_xl/tokenizer.json')

In [ ]:
trainer.model.base_model.save_pretrained('./model_backup/base_mt5_xl')